# Import

In [ ]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Data Load

In [ ]:
# 데이터 분할(폴더) 구분
data_splits = ["train"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    #"신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    #"청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    #"잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    #"채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    #"마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    #"성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']
#months = ['07']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]
        
        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"./open/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

# Data Preprocessing(1) : Concat & Merge

In [ ]:
# 데이터 유형별 설정 
info_categories = ["customer", "sales"]
#info_categories = ["customer", "credit", "sales", "billing", "balance", "channel", "marketing", "performance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

In [ ]:
#### Train ####

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")


customer_train_df = train_dfs["customer_train_df"]
#credit_train_df   = train_dfs["credit_train_df"]
sales_train_df    = train_dfs["sales_train_df"]
#billing_train_df  = train_dfs["billing_train_df"]
#balance_train_df  = train_dfs["balance_train_df"]
#channel_train_df  = train_dfs["channel_train_df"]
#marketing_train_df= train_dfs["marketing_train_df"]
#performance_train_df = train_dfs["performance_train_df"]

gc.collect()

# 저장(최초에는 여기까지 실행하고 그 이후는 밑에부터)

In [ ]:
customer_train_df.to_csv('1merge.csv')
sales_train_df.to_csv('3merge.csv')

# 불러오기(2번째부터는 여기서부터)

In [ ]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
customer_train_df=pd.read_csv('1merge.csv')
sales_train_df=pd.read_csv('3merge.csv')

In [ ]:
#### Train ####

train_df = customer_train_df.merge(sales_train_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: train_step1, shape:", train_df.shape)
del customer_train_df, sales_train_df
gc.collect()



# 저장(최초에는 여기까지 실행하고 그 이후는 밑에부터)

In [ ]:
train_df.to_csv('1_3merge.csv')

# 불러오기(2번째부터는 여기서부터)

In [1]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df=pd.read_csv('1_3merge.csv')

C:\Users\J\AppData\Local\Temp\ipykernel_18688\1154772784.py:1: DtypeWarning: Columns (262) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df=pd.read_csv('1_3merge.csv')


# 전처리

## 결측치 처리

In [3]:
# 연체일자_B0M → 결측은 0, 값이 있으면 1로 변환 후 삭제 // 3번에 해당하지 않은 컬럼이므로 주석처리
#train_df['연체일자_B0M_여부'] = train_df['연체일자_B0M'].notna().astype(int)
#train_df.drop('연체일자_B0M', axis=1, inplace=True)

# 업종 관련 결측 → '없음'으로 채움
train_df['_3순위여유업종'] = train_df['_3순위여유업종'].fillna('없음')
train_df['_2순위여유업종'] = train_df['_2순위여유업종'].fillna('없음')
train_df['_3순위납부업종'] = train_df['_3순위납부업종'].fillna('없음')
train_df['_3순위교통업종'] = train_df['_3순위교통업종'].fillna('없음')
train_df['_1순위여유업종'] = train_df['_1순위여유업종'].fillna('없음')
train_df['_2순위납부업종'] = train_df['_2순위납부업종'].fillna('없음')

# 최종카드론_대출일자 → 결측은 0, 값이 있으면 1로 변환 후 삭제 // 3번에 해당하지 않은 컬럼이므로 주석처리
#train_df['최종카드론_대출여부'] = train_df['최종카드론_대출일자'].notna().astype(int)
#train_df.drop('최종카드론_대출일자', axis=1, inplace=True)

# 최종카드론 코드 결측 → 0으로 채움
train_df['최종카드론_신청경로코드'] = train_df['최종카드론_신청경로코드'].fillna(0)
train_df['최종카드론_금융상환방식코드'] = train_df['최종카드론_금융상환방식코드'].fillna(0)

# RV신청일자 → 결측은 0, 값이 있으면 1로 변환 후 삭제 // 3번에 해당하지 않은 컬럼이므로 주석처리
#train_df['RV신청여부'] = train_df['RV신청일자'].notna().astype(int)
#train_df.drop('RV신청일자', axis=1, inplace=True)

# 나머지 업종 관련 결측 → '없음'으로 채움
train_df['_2순위교통업종'] = train_df['_2순위교통업종'].fillna('없음')
#train_df['OS구분코드'] = train_df['OS구분코드'].fillna('그외') // 3번에 해당하지 않은 컬럼이므로 주석처리
train_df['_3순위쇼핑업종'] = train_df['_3순위쇼핑업종'].fillna('없음')
train_df['_1순위납부업종'] = train_df['_1순위납부업종'].fillna('없음')
train_df['_1순위교통업종'] = train_df['_1순위교통업종'].fillna('없음')
train_df['_2순위쇼핑업종'] = train_df['_2순위쇼핑업종'].fillna('없음')
train_df['_3순위업종'] = train_df['_3순위업종'].fillna('없음')
train_df['_2순위신용체크구분'] = train_df['_2순위신용체크구분'].fillna('없음')
train_df['_1순위쇼핑업종'] = train_df['_1순위쇼핑업종'].fillna('없음')
train_df['_2순위업종'] = train_df['_2순위업종'].fillna('없음')
train_df['_1순위업종'] = train_df['_1순위업종'].fillna('없음')

# 혜택 관련 결측 → 0으로 채움 // 3번에 해당하지 않은 컬럼이므로 주석처리
#train_df['혜택수혜율_B0M'] = train_df['혜택수혜율_B0M'].fillna(0)
#train_df['혜택수혜율_R3M'] = train_df['혜택수혜율_R3M'].fillna(0)

# 최종유효년월_신용_이용 → 결측은 0, 값이 있으면 1로 변환 후 삭제
train_df['최종유효년월_신용_이용_여부'] = train_df['최종유효년월_신용_이용'].notna().astype(int)
train_df.drop('최종유효년월_신용_이용', axis=1, inplace=True)

# 가입통신회사코드, 직장시도명 → 결측 처리
train_df['가입통신회사코드'] = train_df['가입통신회사코드'].fillna('그외')
train_df['직장시도명'] = train_df['직장시도명'].fillna('없음')

# 최종유효년월_신용_이용가능 → 결측은 0, 값이 있으면 1로 변환 후 삭제
train_df['최종유효년월_신용_이용가능_여부'] = train_df['최종유효년월_신용_이용가능'].notna().astype(int)
train_df.drop('최종유효년월_신용_이용가능', axis=1, inplace=True)

# 최종카드발급일자 → 결측은 0, 값이 있으면 1로 변환 후 삭제
train_df['최종카드발급여부'] = train_df['최종카드발급일자'].notna().astype(int)
train_df.drop('최종카드발급일자', axis=1, inplace=True)

# RV전환가능여부, _1순위신용체크구분 → 결측행 제거 // 3번에 해당하지 않은 컬럼이므로 주석처리
#train_df = train_df.dropna(subset=['RV전환가능여부', '_1순위신용체크구분'])


## 기준년월은 datetime type으로 바꾸고 나머지는 10101이라는 값이 있으면 0으로 바꾸고 라벨인코딩하는 코드

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# 기준년월: datetime 변환
train_df['기준년월'] = pd.to_datetime(
    train_df['기준년월'].astype(str).where(train_df['기준년월'].astype(str).str.match(r'^\d{6}$')),
    format='%Y%m',
    errors='coerce'
)
# 날짜 컬럼 리스트 (입회일자 및 최종이용일자 관련)
date_cols = [
    '입회일자_신용', '최종이용일자_기본', '최종이용일자_신판', '최종이용일자_CA',
    '최종이용일자_카드론', '최종이용일자_체크', '최종이용일자_일시불', '최종이용일자_할부'
]
# 10101 이상치 → '0' 문자열로 대체한 후 라벨 인코딩
for col in date_cols:
    # 문자열로 변환하고 이상치 대체
    train_df[col] = train_df[col].astype(str).replace('10101', '0')
    # 라벨 인코딩
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
# 결과 확인 (선택)
print(train_df[date_cols].head())

   입회일자_신용  최종이용일자_기본  최종이용일자_신판  최종이용일자_CA  최종이용일자_카드론  최종이용일자_체크  \
0      259       1109       1090       2933           0       1912   
1      314       1109       1096       2577        1167          0   
2      202       1096       1083       2920         673        793   
3      299       1111       1092       2935           0        744   
4      324        933        920          0           0       2010   

   최종이용일자_일시불  최종이용일자_할부  
0         912       2074  
1         922       1880  
2         909       2058  
3         907       2076  
4         746          0  


## 결측치를 0으로 바꾸고 라벨인코딩 하는 코드

In [ ]:
from sklearn.preprocessing import LabelEncoder
# 처리 대상 컬럼 리스트
# 3번에 없는 컬럼들 이므로 이 부분 생략 
label_date_cols = [
    '연체일자_B0M', '최종카드론_대출일자', 'RV신청일자',
    '최종유효년월_신용_이용', '최종유효년월_신용_이용가능', '최종카드발급일자'
]
# 각 컬럼에 대해 결측치 → '0' 대체 후 라벨 인코딩
for col in label_date_cols:
    # NaN → '0' 문자열로 대체
    train_df[col] = train_df[col].fillna(0).astype(int).astype(str)
    # 라벨 인코딩
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
# 결과 예시 출력
print(train_df[label_date_cols].head())

## float64지만 범주형처럼 보이는 컬럼 처리

In [5]:
# 1. 최종카드론_금융상환방식코드: 결측치 0으로 채우고 category로 변환
train_df['최종카드론_금융상환방식코드'] = train_df['최종카드론_금융상환방식코드'].fillna(0).astype('category')

## int64지만 범주형처럼 보이는 컬럼 처리

In [ ]:
category_cols = [
    '남녀구분코드', '회원여부_이용가능', '회원여부_이용가능_CA',
    '회원여부_이용가능_카드론', '소지여부_신용', '회원여부_연체',
    '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM',
    '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS',
    '마케팅동의여부', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인',
    '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인',
    '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인',
    '일시불ONLY전환가능여부', '연체감액여부_R3M', '시장단기연체여부_R6M',
    '시장단기연체여부_R3M', '청구서발송여부_B0', '청구서발송여부_R3M',
    '청구서발송여부_R6M'
]

In [6]:
from sklearn.preprocessing import LabelEncoder

# 3.승인매출정보 관련 category 컬럼
sales_category_cols = [
    '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론',
    '소지여부_신용', '회원여부_연체', '이용거절여부_카드론',
    '일시불ONLY전환가능여부'
]

# 존재하는 컬럼만 필터링
existing_sales_cols = [col for col in sales_category_cols if col in train_df.columns]

# category 타입으로 변환 후 라벨 인코딩
train_df[existing_sales_cols] = train_df[existing_sales_cols].astype('category')
le = LabelEncoder()
for col in existing_sales_cols:
    train_df[col] = le.fit_transform(train_df[col])


In [1]:
#지금 해야할 것 
# 라벨인코딩 및 원핫 인코딩
# 연령 : 올인원인코딩
# 가입통신회사코드 : 원핫인코딩
# 거주시도명 : 원핫인코딩
# 직장시도명 : 원핫인코딩
# _1순위신용체크구분 : 원핫인코딩
# _2순위신용체크구분 : 원핫인코딩
# 연회비발생카드수_B0M : 올인원인코딩
# 상품관련면제카드수_B0M : 범주가 하나임
# 임직원면제카드수_B0M : 범주가 하나임
# 우수회원면제카드수_B0M : 범주가 하나임
# 기타면제카드수_B0M : 범주가 하나임
# Life_Stage : 원핫인코딩
# 자발한도감액횟수_R12M : 0회,1회 => 0,1
# 한도증액횟수_R12M : 올인원인코딩
# 카드론동의여부 : 원핫인코딩
# RV전환가능여부 : 원핫인코딩
# 한도심사요청건수 : 올인원인코딩
# _1순위업종 : 원핫인코딩
# _2순위업종', '_3순위업종', '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종', '_1순위교통업종', '_2순위교통업종', '_3순위교통업종', '_1순위여유업종', '_2순위여유업종', '_3순위여유업종', '_1순위납부업종', '_2순위납부업종', '_3순위납부업종'
# 최종카드론_신청경로코드 : 원핫인코딩
# 이용금액대 : 올인원인코딩
# 대표결제방법코드 : 범주가 하나임
# 대표청구지고객주소구분코드 : 원핫인코딩
# 대표청구서수령지구분코드 : 원핫인코딩
# 청구서수령방법 : 원핫인코딩
# 할인건수_R3M : 올인원인코딩
# 할인건수_B0M : 올인원인코딩
# 인입횟수_ARS_R6M : 올인원인코딩
# 이용메뉴건수_ARS_R6M : 올인원인코딩
# 방문횟수_PC_R6M : 올인원인코딩
# 방문일수_PC_R6M : 올인원인코딩
# 방문횟수_앱_R6M : 올인원인코딩
# OS구분코드 : 원핫인코딩
# 캠페인접촉건수_R12M : 올인원인코딩
# 캠페인접촉일수_R12M : 올인원인코딩

In [7]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# 라벨 인코딩 대상
all_in_one_cols = [
    '연령', '연회비발생카드수_B0M', '한도증액횟수_R12M', '한도심사요청건수',
    '이용금액대', '할인건수_R3M', '할인건수_B0M', '인입횟수_ARS_R6M',
    '이용메뉴건수_ARS_R6M', '방문횟수_PC_R6M', '방문일수_PC_R6M',
    '방문횟수_앱_R6M', '캠페인접촉건수_R12M', '캠페인접촉일수_R12M'
]

le = LabelEncoder()
for col in all_in_one_cols:
    if col in train_df.columns:
        train_df[col] = le.fit_transform(train_df[col].astype(str))

# 0회/1회 변환
if '자발한도감액횟수_R12M' in train_df.columns:
    train_df['자발한도감액횟수_R12M'] = train_df['자발한도감액횟수_R12M'].map({'0회': 0, '1회': 1}).fillna(0).astype(int)

# 제외 대상
excluded_cols = ['상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', '대표결제방법코드']

# 원핫 인코딩 대상
onehot_cols = [
    '가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분',
    'Life_Stage', '카드론동의여부', 'RV전환가능여부', '_1순위업종', '_2순위업종', '_3순위업종',
    '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종', '_1순위교통업종', '_2순위교통업종', '_3순위교통업종',
    '_1순위여유업종', '_2순위여유업종', '_3순위여유업종', '_1순위납부업종', '_2순위납부업종', '_3순위납부업종',
    '최종카드론_신청경로코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', 'OS구분코드'
]

# 실제 존재하는 컬럼만 필터링
onehot_cols = [col for col in onehot_cols if col in train_df.columns]

# 원핫 인코딩 대상 중 '최종카드론_신청경로코드'가 object 또는 string이 되도록 먼저 변환
train_df['최종카드론_신청경로코드'] = train_df['최종카드론_신청경로코드'].astype(str)

# 다시 실제 존재하는 컬럼만 필터링 (object형으로 존재하면 포함됨)
onehot_cols = [col for col in onehot_cols if col in train_df.columns]

# 원핫 인코딩 재실행
train_df_encoded = pd.get_dummies(train_df, columns=onehot_cols, drop_first=True)

# 결과 확인
print(train_df_encoded.filter(like='최종카드론_신청경로코드').columns)


Index(['최종카드론_신청경로코드_1', '최종카드론_신청경로코드_2', '최종카드론_신청경로코드_7', '최종카드론_신청경로코드_8',
       '최종카드론_신청경로코드_D'],
      dtype='object')


# 다시 저장

In [8]:
train_df.to_csv('1_3merge_final.csv')
train_df_encoded.to_csv('1_3merge_encoded.csv')

# 로드

In [ ]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_df=pd.read_csv('1_3merge_final.csv')
train_df_encoded=pd.read_csv('1_3merge_encoded.csv')

# 상관관계 구하기(train_cols 수정하면서 하기)

In [9]:
import pandas as pd

# Segment를 숫자로 변환
train_df['Segment_code'] = train_df['Segment'].astype('category').cat.codes
train_df_encoded['Segment_code'] = train_df['Segment_code']

# 1. 승인매출 정보 컬럼 리스트 (명세서 No.3 ~ No.406 기준)
target_cols = train_df.columns[
    train_df.columns.get_loc("최종이용일자_기본") : train_df.columns.get_loc("이용금액대") + 1
]

# 2-1. 원본 데이터 내 상관계수 (숫자형 변수만)
target_numeric = train_df[target_cols].select_dtypes(include='number').copy()
target_numeric['Segment_code'] = train_df['Segment_code']
corr_sales = target_numeric.corr()['Segment_code'].drop('Segment_code')

# 2-2. 인코딩 데이터 내 상관계수 (승인매출 변수 중 인코딩된 것만)
encoded_target_cols = [col for col in train_df_encoded.columns if col.startswith(tuple(target_cols))]
corr_encoded_sales = train_df_encoded[encoded_target_cols + ['Segment_code']].corr()['Segment_code'].drop('Segment_code')

# 3. 두 결과를 병합하여 비교 테이블 생성
corr_sales_df = corr_sales.rename("원본(train_df)").to_frame()
corr_encoded_df = corr_encoded_sales.rename("인코딩(train_df_encoded)").to_frame()

combined_corr = pd.merge(
    corr_sales_df,
    corr_encoded_df,
    how='outer',
    left_index=True,
    right_index=True
)

# 절댓값 기준 정렬
combined_corr['최대상관계수(절댓값)'] = combined_corr.abs().max(axis=1)
combined_corr = combined_corr.sort_values(by='최대상관계수(절댓값)', ascending=False)


In [11]:
# pandas 출력 옵션 변경 - 모든 행 보기
pd.set_option('display.max_rows', None)

# 전체 출력
print(combined_corr)

                       원본(train_df)  인코딩(train_df_encoded)  최대상관계수(절댓값)
정상청구원금_B5M                -0.660738              -0.660738     0.660738
정상청구원금_B0M                -0.634571              -0.634571     0.634571
정상청구원금_B2M                -0.628078              -0.628078     0.628078
이용금액_일시불_R12M             -0.595858              -0.595858     0.595858
이용금액_일시불_B0M              -0.575032              -0.575032     0.575032
이용금액_일시불_R6M              -0.568893              -0.568893     0.568893
이용금액_일시불_R3M              -0.567357              -0.567357     0.567357
이용금액_오프라인_B0M             -0.566186              -0.566186     0.566186
정상입금원금_B5M                -0.551228              -0.551228     0.551228
정상입금원금_B0M                -0.549280              -0.549280     0.549280
이용금액_오프라인_R3M             -0.539827              -0.539827     0.539827
이용금액_오프라인_R6M             -0.538775              -0.538775     0.538775
정상입금원금_B2M                -0.536208              -0.536208     0

In [12]:
# 1. 상관계수 0.1 이상인 칼럼 이름 추출
high_corr_cols = combined_corr[combined_corr['최대상관계수(절댓값)'] >= 0.1].index

# 2. 원본 train_df의 컬럼 목록
original_cols = set(train_df.columns)

# 3. 원본 컬럼 중 상관관계 높은 컬럼만 추출
high_corr_original_cols = [col for col in high_corr_cols if col in original_cols]

# 결과 출력
print("✅ 인코딩 전 원본 컬럼 중 Segment와 상관관계가 0.1 이상인 변수 개수:", len(high_corr_original_cols))
print(high_corr_original_cols)


✅ 인코딩 전 원본 컬럼 중 Segment와 상관관계가 0.1 이상인 변수 개수: 245
['정상청구원금_B5M', '정상청구원금_B0M', '정상청구원금_B2M', '이용금액_일시불_R12M', '이용금액_일시불_B0M', '이용금액_일시불_R6M', '이용금액_일시불_R3M', '이용금액_오프라인_B0M', '정상입금원금_B5M', '정상입금원금_B0M', '이용금액_오프라인_R3M', '이용금액_오프라인_R6M', '정상입금원금_B2M', '_3순위업종_이용금액', '_2순위업종_이용금액', '_2순위쇼핑업종_이용금액', '최대이용금액_일시불_R12M', '이용건수_신용_R12M', '이용건수_신판_R12M', '이용건수_일시불_R12M', '_1순위업종_이용금액', '이용금액대', '_3순위쇼핑업종_이용금액', '이용건수_오프라인_B0M', '이용가맹점수', '쇼핑_도소매_이용금액', '이용건수_오프라인_R6M', '이용건수_오프라인_R3M', '이용건수_신용_B0M', '이용건수_신용_R6M', '이용건수_신용_R3M', '이용건수_신판_R6M', '이용건수_신판_B0M', '이용건수_신판_R3M', '이용건수_일시불_B0M', '이용건수_일시불_R6M', '이용건수_일시불_R3M', '_1순위교통업종_이용금액', '연체입금원금_B0M', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '이용금액_온라인_B0M', '교통_주유이용금액', '연체입금원금_B2M', '연체입금원금_B5M', '이용금액_페이_온라인_B0M', '_1순위쇼핑업종_이용금액', '이용건수_온라인_R6M', '이용금액_페이_온라인_R3M', '이용금액_간편결제_B0M', '이용건수_온라인_R3M', '이용금액_페이_온라인_R6M', '연속유실적개월수_기본_24M_카드', '이용건수_온라인_B0M', '이용금액_온라인_R3M', '이용금액_할부_R12M', '쇼핑_온라인_이용금액', '이용금액_온라인_R6M', '_2순위교통업종_이용금액', '이용금액_할부_무이자_R12M', 

In [13]:
cols = ['소지카드수_유효_신용',
 '소지카드수_이용가능_신용',
 '입회일자_신용',
 '입회경과개월수_신용',
 '유효카드수_신용체크',
 '유효카드수_신용',
 '유효카드수_신용_가족',
 '유효카드수_체크',
 '이용가능카드수_신용체크',
 '이용가능카드수_신용',
 '이용가능카드수_신용_가족',
 '이용가능카드수_체크',
 '이용카드수_신용체크',
 '이용카드수_신용',
 '이용카드수_신용_가족',
 '이용카드수_체크',
 '이용금액_R3M_신용체크',
 '이용금액_R3M_신용',
 '이용금액_R3M_신용_가족',
 '이용금액_R3M_체크',
 '_1순위카드이용금액',
 '_1순위카드이용건수',
 '_2순위카드이용금액',
 '_2순위카드이용건수',
 '최종유효년월_신용_이용',
 '최초한도금액',
 '카드이용한도금액',
 'CA한도금액',
 'CA이자율_할인전',
 '카드이용한도금액_B1M',
'수신거부여부_TM',
'수신거부여부_DM',
'수신거부여부_메일',
'보유여부_해외겸용_본인',
'이용가능여부_해외겸용_본인',
'이용여부_3M_해외겸용_본인',
'보유여부_해외겸용_신용_본인',
 '이용가능여부_해외겸용_신용_본인',
 '이용여부_3M_해외겸용_신용_본인',
'직장시도명',
 '_2순위신용체크구분'
'정상청구원금_B5M', '정상청구원금_B0M', '정상청구원금_B2M', '이용금액_일시불_R12M', '이용금액_일시불_B0M', '이용금액_일시불_R6M', '이용금액_일시불_R3M', 
        '이용금액_오프라인_B0M', '정상입금원금_B5M', '정상입금원금_B0M', '이용금액_오프라인_R3M', '이용금액_오프라인_R6M', '정상입금원금_B2M', 
        '_3순위업종_이용금액', '_2순위업종_이용금액', '_2순위쇼핑업종_이용금액', '최대이용금액_일시불_R12M', '이용건수_신용_R12M', '이용건수_신판_R12M', 
        '이용건수_일시불_R12M', '_1순위업종_이용금액', '이용금액대', '_3순위쇼핑업종_이용금액', '이용건수_오프라인_B0M', '이용가맹점수', '쇼핑_도소매_이용금액', 
        '이용건수_오프라인_R6M', '이용건수_오프라인_R3M', '이용건수_신용_B0M', '이용건수_신용_R6M', '이용건수_신용_R3M', '이용건수_신판_R6M', 
        '이용건수_신판_B0M', '이용건수_신판_R3M', '이용건수_일시불_B0M', '이용건수_일시불_R6M', '이용건수_일시불_R3M', '_1순위교통업종_이용금액', 
        '연체입금원금_B0M', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '이용금액_온라인_B0M', '교통_주유이용금액', '연체입금원금_B2M', '연체입금원금_B5M', 
        '이용금액_페이_온라인_B0M', '_1순위쇼핑업종_이용금액', '이용건수_온라인_R6M', '이용금액_페이_온라인_R3M', '이용금액_간편결제_B0M', '이용건수_온라인_R3M', 
        '이용금액_페이_온라인_R6M', '연속유실적개월수_기본_24M_카드', '이용건수_온라인_B0M', '이용금액_온라인_R3M', '이용금액_할부_R12M', '쇼핑_온라인_이용금액', 
        '이용금액_온라인_R6M', '_2순위교통업종_이용금액', '이용금액_할부_무이자_R12M', '이용금액_간편결제_R3M', '이용건수_간편결제_R6M', 
        '이용건수_페이_온라인_R6M', '교통_정비이용금액', '할부금액_3M_R12M', '이용건수_간편결제_R3M', '최대이용금액_할부_무이자_R12M', '최대이용금액_할부_R12M', 
        '이용금액_간편결제_R6M', '이용건수_페이_온라인_R3M', '이용건수_간편결제_B0M', '_3순위교통업종_이용금액', '쇼핑_편의점_이용금액', 
        '이용건수_페이_온라인_B0M', '할부금액_무이자_3M_R12M', '이용개월수_온라인_R6M', '이용개월수_페이_온라인_R6M', '납부_기타이용금액', 
        '이용개월수_간편결제_R6M', '최대이용금액_CA_R12M', '할부금액_6M_R12M', '쇼핑_기타_이용금액', '이용금액_해외', '이용개월수_할부_무이자_R12M', 
        '이용금액_할부_무이자_R6M', '이용개월수_할부_R12M', '이용건수_할부_R12M', '이용금액_할부_R6M', '이용건수_할부_무이자_R12M', '_1순위여유업종_이용금액', 
        '이용금액_CA_R12M', '할부건수_3M_R12M', '쇼핑_백화점_이용금액', '할부건수_무이자_3M_R12M', '할부금액_무이자_6M_R12M', '이용건수_CA_R12M', 
        '이용금액_CA_R6M', '이용개월수_신용_R12M', '이용개월수_신판_R12M', '이용개월수_일시불_R12M', '이용개월수_CA_R12M', '이용금액_CA_R3M', 
        '이용금액_CA_B0M', '이용건수_할부_R6M', '이용후경과월_할부_무이자', '이용개월수_할부_R6M', '이용금액_할부_무이자_R3M', '이용개월수_할부_무이자_R6M', 
        '이용건수_할부_무이자_R6M', '교통_택시이용금액', '최종이용일자_할부', '이용금액_할부_R3M', '이용후경과월_할부', '최종이용일자_CA', '이용건수_CA_R6M', 
        '여유_운동이용금액', '이용개월수_CA_R6M', '이용건수_할부_R3M', '이용건수_할부_무이자_R3M', '이용개월수_할부_R3M', '이용개월수_할부_무이자_R3M', 
        '할부건수_6M_R12M', '이용후경과월_CA', '이용건수_할부_B0M', '이용개월수_전체_R6M', '이용개월수_CA_R3M', '이용건수_할부_무이자_B0M', 
        '_1순위납부업종_이용금액', '이용금액_할부_무이자_B0M', '이용건수_CA_R3M', '할부건수_무이자_6M_R12M', '이용금액_할부_B0M', '이용금액_체크_R12M', 
        '이용건수_CA_B0M', '이용금액_체크_R6M', '이용금액_체크_R3M', '이용개월수_전체_R3M', '이용개월수_오프라인_R6M', '이용건수_체크_R12M', 
        '이용개월수_신용_R6M', '이용금액_체크_B0M', 'RP건수_B0M', '최대이용금액_체크_R12M', '이용건수_체크_R6M', '_2순위여유업종_이용금액', 
        '이용건수_체크_R3M', '이용개월수_신판_R6M', '이용개월수_일시불_R6M', '이용건수_체크_B0M', '이용후경과월_신용', '이용개월수_신용_R3M', 
        '_2순위납부업종_이용금액', '이용개월수_신판_R3M', '이용개월수_일시불_R3M', '최대이용금액_할부_유이자_R12M', '이용금액_할부_유이자_R12M', 
        '이용후경과월_신판', '납부_통신비이용금액', 'RP유형건수_B0M', '할부금액_유이자_3M_R12M', '교통_철도버스이용금액', '납부_보험료이용금액', 
        'RP후경과월_교통', 'RP건수_통신_B0M', '쇼핑_아울렛_이용금액', '이용후경과월_일시불', '이용개월수_할부_유이자_R12M', '이용개월수_결제일_R6M', 
        '이용개월수_체크_R12M', '신청건수_ATM_CA_R6M', '할부금액_유이자_6M_R12M', '이용후경과월_할부_유이자', '연속무실적개월수_기본_24M_카드', 
        '이용건수_할부_유이자_R12M', '선입금원금_B5M', '여유_숙박이용금액', '여유_공연이용금액', '이용개월수_결제일_R3M', '선입금원금_B0M', 
        '이용개월수_체크_R6M', '할부건수_유이자_3M_R12M', '선입금원금_B2M', 'RP건수_교통_B0M', '최종이용일자_기본', '최종이용일자_일시불', 
        '이용금액_선결제_R6M', '최종이용일자_체크', 'RP후경과월_통신', '이용개월수_체크_R3M', '카드론이용월수_누적', '최종이용일자_신판', 
        '카드론이용건수_누적', '이용후경과월_체크', '이용금액_A페이_B0M', '이용금액_페이_오프라인_B0M', '이용금액_A페이_R3M', '이용금액_페이_오프라인_R3M', 
        'RP후경과월', '이용건수_A페이_R6M', '이용금액_선결제_R3M', 'RP금액_B0M', '최종카드론_대출이율', '이용개월수_페이_오프라인_R6M', '이용횟수_선결제_R6M', 
        '이용금액_A페이_R6M', '이용금액_할부_유이자_R6M', '이용금액_페이_오프라인_R6M', '교통_버스지하철이용금액', '신청건수_ATM_CA_B0', '이용개월수_A페이_R6M', 
        '이용건수_페이_오프라인_R6M', '이용건수_A페이_R3M', '이용건수_할부_유이자_R6M', '이용건수_페이_오프라인_B0M', '이용건수_페이_오프라인_R3M', 
        '_3순위여유업종_이용금액', '이용개월수_할부_유이자_R6M', '이용금액_선결제_B0M', '이용금액_C페이_B0M', '이용금액_C페이_R3M', '이용건수_A페이_B0M', 
        '이용건수_C페이_B0M', '_3순위납부업종_이용금액', '이용개월수_선결제_R6M', '최초카드론이용경과월', '이용횟수_선결제_R3M', '이용금액_C페이_R6M', 
        '이용건수_C페이_R3M', '여유_기타이용금액', '최종카드론이용경과월', '이용개월수_할부_유이자_R3M', '이용건수_선결제_R6M', '할부건수_유이자_6M_R12M', 
        '이용횟수_선결제_B0M', '이용건수_할부_유이자_R3M', '최종이용일자_카드론', '이용금액_할부_유이자_R3M', '이용건수_C페이_R6M', '할부금액_12M_R12M', 
        '이용개월수_C페이_R6M', '카드론이용금액_누적', '할부금액_무이자_12M_R12M', '이용건수_할부_유이자_B0M', '이용금액_D페이_R6M', '이용건수_선결제_R3M',
    '청구금액_B0',
 '청구금액_R3M',
 '청구금액_R6M',
 '포인트_마일리지_건별_B0M',
 '포인트_마일리지_건별_R3M',
 '포인트_포인트_건별_B0M',
 '포인트_포인트_건별_R3M',
 '포인트_마일리지_월적립_B0M',
 '포인트_마일리지_월적립_R3M',
 '포인트_포인트_월적립_B0M',
 '포인트_포인트_월적립_R3M',
 '포인트_적립포인트_R12M',
 '포인트_적립포인트_R3M',
 '포인트_이용포인트_R12M',
 '포인트_이용포인트_R3M',
 '마일_적립포인트_R12M',
 '마일_적립포인트_R3M',
 '마일_이용포인트_R12M',
 '마일_이용포인트_R3M',
 '마일_잔여포인트_B0M',
 '할인금액_R3M',
 '할인금액_B0M',
 '할인금액_청구서_R3M',
 '할인금액_청구서_B0M',
 '상환개월수_결제일_R6M',
 '상환개월수_결제일_R3M',
 '선결제건수_R6M',
 '선결제건수_R3M',
 '혜택수혜금액_R3M',
 '포인트_마일리지_환산_B0M',
 '혜택수혜금액',
 '잔액_일시불_B0M',
 '잔액_할부_B0M',
 '잔액_현금서비스_B0M',
 '월중평잔_일시불_B0M',
 '월중평잔_할부_B0M',
 '월중평잔_CA_B0M',
 '잔액_현금서비스_B1M',
 '잔액_현금서비스_B2M',
 '잔액_할부_B1M',
 '잔액_할부_B2M',
 '잔액_일시불_B1M',
 '잔액_일시불_B2M',
 '잔액_할부_유이자_B0M',
 '잔액_할부_무이자_B0M',
 '월중평잔',
 '월중평잔_일시불',
 '월중평잔_RV일시불',
 '월중평잔_할부',
 '월중평잔_CA',
 '평잔_3M',
 '평잔_일시불_3M',
 '평잔_일시불_해외_3M',
 '평잔_RV일시불_3M',
 '평잔_RV일시불_해외_3M',
 '평잔_할부_3M',
 '평잔_CA_3M',
 '평잔_6M',
 '평잔_일시불_6M',
 '평잔_일시불_해외_6M',
 '평잔_RV일시불_6M',
 '평잔_RV일시불_해외_6M',
 '평잔_할부_6M',
 '평잔_CA_6M',
 '인입횟수_ARS_R6M',
 '인입일수_ARS_R6M',
 '인입월수_ARS_R6M',
 '인입횟수_ARS_B0M',
 '이용메뉴건수_ARS_B0M',
 '인입일수_ARS_B0M',
 '방문월수_PC_R6M',
 '방문후경과월_PC_R6M',
 '방문횟수_앱_R6M',
 '방문일수_앱_R6M',
 '방문월수_앱_R6M',
 '방문후경과월_앱_R6M',
 '방문횟수_PC_B0M',
 '방문일수_PC_B0M',
 '방문횟수_앱_B0M',
 '방문일수_앱_B0M',
 '인입횟수_IB_R6M',
 '인입일수_IB_R6M',
 '인입월수_IB_R6M',
 '이용메뉴건수_IB_R6M',
 '인입후경과월_IB_R6M',
 '상담건수_R6M',
 'IB문의건수_사용승인내역_R6M',
 'IB문의건수_선결제_R6M',
 '불만제기후경과월_R12M',
 '당사멤버쉽_방문월수_R6M',
 '홈페이지_금융건수_R6M',
 '홈페이지_선결제건수_R6M',
 '홈페이지_금융건수_R3M',
 '홈페이지_선결제건수_R3M',
 'OS구분코드',
 '청구서발송여부_B0',
 '청구서발송여부_R3M',
 '청구서발송여부_R6M',
        '컨택건수_이용유도_TM_R6M',
    '변동률_RVCA평잔',
    '변동률_RV일시불평잔',
    '컨택건수_이용유도_EM_B0M',
    '컨택건수_이용유도_EM_R6M',
    '잔액_신판평균한도소진율_r6m',
    '잔액_신판평균한도소진율_r3m',
    '잔액_신판최대한도소진율_r3m',
    '잔액_신판최대한도소진율_r6m',
    '잔액_신판ca평균한도소진율_r6m',
    '잔액_신판ca평균한도소진율_r3m',
    '잔액_신판ca최대한도소진율_r3m',
    '잔액_신판ca최대한도소진율_r6m']

In [14]:
print(len(cols))

396
